# 뉴스 기사 GraphRAG 검색 시스템

Neo4j 그래프 데이터베이스에 저장된 뉴스 기사를 3가지 방식으로 검색하고,
**ToolsRetriever** 기반 GraphRAG 파이프라인을 구축합니다.

## 구성 요소

| 구분 | 서비스 | 모델 |
|------|--------|------|
| **LLM** | Ollama (로컬) | `qwen3:8b-q4_K_M` |
| **Embedding** | Ollama (로컬) | `bona/bge-m3-korean` (1024차원) |
| **Graph DB** | Neo4j | `bolt://127.0.0.1:7687` |

## 검색 방식

1. **Vector** — Content 노드 벡터 유사도 검색 (의미 기반)
2. **VectorCypher** — 벡터 검색 + Cypher 확장 (같은 카테고리 관련 기사)
3. **Text2Cypher** — 자연어 → Cypher 변환 (구조적 검색)

## 그래프 스키마

```
(Media)-[:PUBLISHED]->(Article)-[:HAS_CHUNK]->(Content)
                          │
                     [:BELONGS_TO]
                          │
                      (Category)
```

## 0. 환경 설정

`.env` 파일에서 설정을 로드하고, Ollama LLM / Embedder / Neo4j 드라이버를 초기화합니다.

```
# .env 파일 예시
NEO4J_URI=bolt://127.0.0.1:7687
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=your_password
NEO4J_DB=neo4j

# Ollama 로컬 LLM/Embedding
OLLAMA_MODEL=qwen3:8b-q4_K_M
OLLAMA_EMBEDDING_MODEL=bona/bge-m3-korean
```

> 사전 준비: `ollama pull qwen3:8b-q4_K_M` 및 `ollama pull bona/bge-m3-korean` 실행 필요

In [ ]:
import os, json, warnings, dotenv, neo4j
dotenv.load_dotenv(dotenv_path=os.path.join(os.path.dirname(os.path.abspath("")), ".env"), override=True)
warnings.filterwarnings("ignore")

from neo4j_graphrag.llm import OllamaLLM
from neo4j_graphrag.embeddings import OllamaEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever, Text2CypherRetriever, ToolsRetriever
from neo4j_graphrag.indexes import create_vector_index
from neo4j_graphrag.generation import RagTemplate, GraphRAG

print("모듈 로드 완료")

In [ ]:
# ── 초기화 + 연결 체크 ──
OLLAMA_MODEL     = os.getenv("OLLAMA_MODEL", "qwen3:8b-q4_K_M")
EMBEDDING_MODEL  = os.getenv("OLLAMA_EMBEDDING_MODEL", "bona/bge-m3-korean")
EMBEDDING_DIMENSION = 1024

NEO4J_URI  = os.getenv("NEO4J_URI", "bolt://127.0.0.1:7687")
NEO4J_AUTH = (os.getenv("NEO4J_USERNAME", "neo4j"), os.getenv("NEO4J_PASSWORD", "neo4j"))
NEO4J_DB   = os.getenv("NEO4J_DB", "neo4j")

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
llm = OllamaLLM(
    model_name=OLLAMA_MODEL,
    model_params={
        "options": {"temperature": 0.1, "num_predict": 4096, "num_ctx": 8192, "repeat_penalty": 1.2, "repeat_last_n": 128},
        "think": False,
    },
)
embedder = OllamaEmbeddings(model=EMBEDDING_MODEL)

# 연결 체크
driver.verify_connectivity()
llm.invoke("OK")
embedder.embed_query("test")

print(f"Neo4j:     {NEO4J_URI} (db: {NEO4J_DB})")
print(f"LLM:       {OLLAMA_MODEL}")
print(f"Embedding: {EMBEDDING_MODEL} ({EMBEDDING_DIMENSION}차원)")
print("All OK")

---
## 1. Vector Retriever

Content 노드의 `embedding` 속성을 이용한 **벡터 유사도 검색**입니다.

- 검색 대상: `Content.chunk` (기사 본문 청크)
- 유사도: cosine
- 차원: 1024 (bge-m3-korean)

### 1-1. Content 노드에 임베딩 생성 (최초 1회)

In [ ]:
# 기존 임베딩/인덱스 삭제 후 bge-m3-korean으로 재생성합니다
with driver.session(database=NEO4J_DB) as session:
    # 기존 임베딩 & 벡터 인덱스 정리 (차원이 다르므로 삭제 필수)
    session.run("MATCH (c:Content) REMOVE c.embedding")
    session.run("DROP INDEX content_vector_index IF EXISTS")
    print("기존 임베딩/인덱스 삭제 완료")

    records = session.run(
        "MATCH (c:Content) RETURN elementId(c) AS id, c.chunk AS text"
    ).data()

    if records:
        print(f"임베딩 생성 대상: {len(records)}개 Content 노드")
        for i, rec in enumerate(records):
            vec = embedder.embed_query(rec["text"])
            session.run(
                "MATCH (c) WHERE elementId(c) = $id SET c.embedding = $embedding",
                {"id": rec["id"], "embedding": vec},
            )
            if (i + 1) % 20 == 0:
                print(f"  {i + 1}/{len(records)} 완료")
        print(f"임베딩 생성 완료! ({len(records)}개, {EMBEDDING_DIMENSION}차원)")
    else:
        print("Content 노드가 없습니다. GraphBuilder를 먼저 실행하세요.")

### 1-2. 벡터 인덱스 생성 + VectorRetriever

In [ ]:
INDEX_NAME = "content_vector_index"

create_vector_index(
    driver,
    INDEX_NAME,
    label="Content",
    embedding_property="embedding",
    dimensions=EMBEDDING_DIMENSION,
    similarity_fn="cosine",
    neo4j_database=NEO4J_DB,
)
print(f"벡터 인덱스 '{INDEX_NAME}' 생성/확인 완료")

vector_retriever = VectorRetriever(
    driver=driver,
    index_name=INDEX_NAME,
    embedder=embedder,
    neo4j_database=NEO4J_DB,
)

### 1-3. Vector 검색 테스트

In [ ]:
results = vector_retriever.search(query_text="북한", top_k=3)

print(f"검색 결과 수: {len(results.items)}")
for i, item in enumerate(results.items):
    print(f"\n결과 {i+1}:")
    print(f"  Content: {str(item.content)[:200]}...")
    if item.metadata:
        print(f"  Score: {item.metadata.get('score', 'N/A')}")

---
## 2. VectorCypher Retriever

벡터 검색으로 찾은 Content → Article을 기준으로,
**같은 카테고리의 관련 기사**를 Cypher 쿼리로 함께 반환합니다.

```cypher
MATCH (content:Content)<-[:HAS_CHUNK]-(article:Article)
OPTIONAL MATCH (article)-[:BELONGS_TO]->(category)<-[:BELONGS_TO]-(related:Article)
RETURN content, article, category, related
```

In [ ]:
RETRIEVAL_QUERY = """
WITH node AS content, score
MATCH (content)<-[:HAS_CHUNK]-(article:Article)
OPTIONAL MATCH (article)-[:BELONGS_TO]->(category:Category)
OPTIONAL MATCH (category)<-[:BELONGS_TO]-(related_article:Article)
WHERE related_article <> article

RETURN
    content.content_id AS content_id,
    content.chunk AS chunk,
    content.title AS content_title,
    article.article_id AS article_id,
    article.title AS article_title,
    article.url AS article_url,
    article.published_date AS article_date,
    category.name AS category_name,
    score AS similarity_score,
    collect(DISTINCT {
        article_id: related_article.article_id,
        title: related_article.title,
        url: related_article.url,
        published_date: related_article.published_date
    })[0..5] AS related_articles
"""

vector_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name=INDEX_NAME,
    retrieval_query=RETRIEVAL_QUERY,
    embedder=embedder,
    neo4j_database=NEO4J_DB,
)
print("VectorCypherRetriever 생성 완료")

### 2-1. VectorCypher 검색 테스트

In [ ]:
results = vector_cypher_retriever.search(query_text="삼성전자")

print(f"검색 결과 수: {len(results.items)}")
for i, item in enumerate(results.items):
    print(f"\n결과 {i+1}:")
    print(f"  Content: {str(item.content)[:300]}...")

---
## 3. Text2Cypher Retriever

자연어 질문을 **LLM이 Cypher 쿼리로 변환**하여 Neo4j에서 구조적 검색을 수행합니다.

- 스키마 정보를 자동 추출하여 LLM에게 제공
- Few-shot 예제로 정확도 향상

### 3-1. 스키마 추출

In [ ]:
def get_schema():
    """Neo4j 데이터베이스의 스키마 정보를 추출합니다"""
    with driver.session(database=NEO4J_DB) as session:
        node_info = session.run("""
            CALL db.schema.nodeTypeProperties()
            YIELD nodeType, propertyName, propertyTypes
            RETURN nodeType, collect(propertyName) as properties
        """).data()

        patterns = session.run("""
            MATCH (n)-[r]->(m)
            RETURN DISTINCT labels(n)[0] as source, type(r) as rel, labels(m)[0] as target
            LIMIT 20
        """).data()

    schema = "=== Neo4j Schema ===\n\n노드 타입:\n"
    for n in node_info:
        schema += f"- {n['nodeType']}: {n['properties']}\n"
    schema += "\n관계 패턴:\n"
    for p in patterns:
        schema += f"- ({p['source']})-[:{p['rel']}]->({p['target']})\n"
    return schema

neo4j_schema = get_schema()
print(neo4j_schema)

### 3-2. Text2Cypher Retriever 생성

In [ ]:
CYPHER_EXAMPLES = [
    """
    USER INPUT: 경제 분야의 최신 뉴스 알려주세요
    CYPHER QUERY:
    MATCH (a:Article)-[:BELONGS_TO]->(c:Category {name: \"경제\"})
    RETURN a.article_id, a.title, a.url, a.published_date
    ORDER BY a.published_date DESC LIMIT 10
    """,
    """
    USER INPUT: 매일경제에서 나온 최신 뉴스 3개 보여주세요
    CYPHER QUERY:
    MATCH (m:Media {name: \"매일경제\"})-[:PUBLISHED]->(a:Article)
    RETURN a.article_id, a.title, a.url, a.published_date
    ORDER BY a.published_date DESC LIMIT 3
    """,
    """
    USER INPUT: 카테고리별 기사 개수를 알려주세요
    CYPHER QUERY:
    MATCH (a:Article)-[:BELONGS_TO]->(c:Category)
    RETURN c.name as category, count(a) as article_count
    ORDER BY article_count DESC
    """,
]

text2cypher_retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema=neo4j_schema,
    examples=CYPHER_EXAMPLES,
    neo4j_database=NEO4J_DB,
)
print("Text2CypherRetriever 생성 완료")

### 3-3. Text2Cypher 검색 테스트

In [ ]:
results = text2cypher_retriever.search(query_text="정치 카테고리의 최신 기사 5개를 보여주세요")

for j, item in enumerate(results.items):
    print(f"결과 {j+1}: {item.content}")

---
## 4. ToolsRetriever (통합 검색)

3개의 Retriever를 **Tool**로 변환하고, LLM이 질문에 따라 **적절한 도구를 자동 선택**합니다.

| Tool | 용도 |
|------|------|
| `vector_retriever` | 기사 본문 내용 기반 의미 검색 |
| `vectorcypher_retriever` | 기사 상세정보 + 같은 카테고리 관련 기사 |
| `text2cypher_retriever` | 언론사별, 분야별 등 구조적 검색 |

참고: [Neo4j GraphRAG ToolsRetriever 위키독스](https://wikidocs.net/307512)

In [ ]:
# Retriever → Tool 변환
vector_tool = vector_retriever.convert_to_tool(
    name="vector_retriever",
    description=(
        "뉴스 기사 본문의 의미를 기반으로 유사한 내용을 검색한다. "
        "특정 주제, 인물, 사건에 대한 기사를 찾을 때 사용한다. "
        "카테고리 목록, 기사 수 집계, 언론사별 조회에는 사용하지 않는다."
    ),
)
vector_cypher_tool = vector_cypher_retriever.convert_to_tool(
    name="vectorcypher_retriever",
    description=(
        "의미 기반 검색 결과에 기사의 상세정보(제목, URL, 날짜)와 "
        "같은 카테고리의 관련 기사를 함께 반환한다. "
        "특정 주제의 기사를 찾되 관련 기사도 함께 필요할 때 사용한다. "
        "카테고리 목록, 기사 수 집계, 언론사별 조회에는 사용하지 않는다."
    ),
)
text2cypher_tool = text2cypher_retriever.convert_to_tool(
    name="text2cypher_retriever",
    description=(
        "카테고리별 기사 목록, 언론사별 기사 수, 최신 기사 N개 등 "
        "구조적 조건으로 검색한다. "
        "'카테고리', '최신', '목록', '개수', '언론사별' 같은 "
        "조건이 포함된 질문에 반드시 이 도구를 사용한다."
    ),
)

# ToolsRetriever 생성
tools_retriever = ToolsRetriever(
    driver=driver,
    llm=llm,
    tools=[vector_tool, vector_cypher_tool, text2cypher_tool],
)
print("ToolsRetriever 생성 완료 (3개 도구 등록)")

### 4-1. ToolsRetriever 검색 테스트

In [ ]:
results = tools_retriever.search("정치 카테고리의 최신 뉴스 알려줘")

print(f"검색 결과: {len(results.items)}개")
for i, item in enumerate(results.items, 1):
    tool_name = item.metadata.get('tool', 'unknown') if item.metadata else 'unknown'
    print(f"\n[{i}] (tool: {tool_name})")
    print(f"    {str(item.content)[:200]}")

---
## 5. GraphRAG 파이프라인

**ToolsRetriever + LLM**을 결합한 최종 RAG 파이프라인입니다.

```
사용자 질문 → ToolsRetriever(도구 자동 선택) → 검색 결과(Context) → LLM → 최종 답변
```

In [20]:
prompt_template = RagTemplate(
    template="""당신은 Neo4j 그래프 데이터베이스에 저장된 뉴스 기사만을 검색하여 답변하는 어시스턴트입니다.

질문: {query_text}

검색된 문서 정보:
{context}

반드시 지켜야 할 규칙:
1. 오직 위의 검색 결과(Context)에 포함된 내용만 사용하여 답변하세요.
2. 검색 결과가 비어있거나 질문과 관련 없는 경우, "해당 내용의 기사가 데이터베이스에 없습니다."라고만 답하세요.
3. 절대로 검색 결과에 없는 내용을 추측하거나 자체 지식으로 답변하지 마세요.
4. 검색된 기사를 목록으로 정리하되, 같은 기사(같은 URL)는 한 번만 표시하세요.
5. 각 기사의 제목, URL, 발행일을 반드시 포함하세요.

답변:""",
    expected_inputs=["context", "query_text"],
)

graphrag = GraphRAG(
    llm=llm,
    retriever=tools_retriever,
    prompt_template=prompt_template,
)
print("GraphRAG 파이프라인 생성 완료")

GraphRAG 파이프라인 생성 완료


### 5-1. GraphRAG 질의 테스트

In [21]:
def ask(question):
    """GraphRAG에 질문하고 결과를 출력합니다"""
    result = graphrag.search(query_text=question, return_context=True)

    print(f"\n{'='*60}")
    print(f"Q: {question}")
    print(f"{'='*60}")
    print(result.answer)

    if hasattr(result, 'retriever_result') and result.retriever_result:
        tools_used = set()
        for item in result.retriever_result.items:
            if item.metadata:
                tools_used.add(item.metadata.get('tool', 'unknown'))
        print(f"\n사용된 도구: {', '.join(tools_used)}")
        print(f"검색 결과 수: {len(result.retriever_result.items)}개")
    print()

In [ ]:
ask("정치 카테고리의 최신 기사")

In [23]:
ask("트럼프 대통령 관련 내용")


Q: 트럼프 대통령 관련 내용
해당 내용의 기사가 데이터베이스에 없습니다.

사용된 도구: 
검색 결과 수: 0개



In [22]:
ask("삼성전자 승진 관련 기사")


Q: 삼성전자 승진 관련 기사
해당 내용의 기사가 데이터베이스에 없습니다.

사용된 도구: 
검색 결과 수: 0개



In [24]:
ask("언론사별 기사 개수")


Q: 언론사별 기사 개수
언론사별 기사 개수는 다음과 같습니다:

- YTN: 4개
- SBS: 3개
- 국민일보: 3개
- 뉴스1: 3개
- 전자신문: 3개
- 지디넷코리아: 3개
- 한국경제: 2개
- TV조선: 2개
- 세계일보: 2개
- 서울경제: 2개
- KBS: 2개
- 아시아경제: 2개
- 연합뉴스TV: 2개
- 서울신문: 2개
- 데일리안: 2개
- 프레시안: 1개
- CJB청주방송: 1개
- 경향신문: 1개
- 파이낸셜뉴스: 1개
- 매일경제: 1개
- 헤럴드경제: 1개
- 한국경제TV: 1개
- 디지털타임스: 1개
- 조선일보: 1개
- 이데일리: 1개
- 노컷뉴스: 1개
- 조선비즈: 1개
- 연합뉴스: 1개
- 시사저널: 1개
- 매일신문: 1개
- 코메디닷컴: 1개
- 헬스조선: 1개
- 뉴시스: 1개
- 디지털데일리: 1개
- JTBC: 1개
- 채널A: 1개
- 머니투데이: 1개
- 동아일보: 1개

기사 제목, URL, 발행일 등은 제공된 검색 결과에 포함되지 않았으므로 제공할 수 없습니다.

사용된 도구: text2cypher_retriever
검색 결과 수: 38개

